In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import backtrader as bt
import tushare as ts
from datetime import datetime
import tushare as ts


#plt.rcParams['figure.figsize'] = (16, 12)
#plt.rcParams['figure.dpi'] = 300

In [9]:
ts.set_token('5d3c09f16bc3ad409898ff6cfc10aadeeae28ba23aa240c047dd125c')
pro = ts.pro_api()

start_date = '20170101'
end_date = '20191201'
code = '600519.SH'

def get_data(code = code,start_date = start_date,end_date = end_date):
    df = pro.daily(ts_code = code,start_date = start_date,end_date = end_date)
    df = df.iloc[::-1] #序列倒转
    df.index = pd.to_datetime(df.trade_date) #index改为时间格式
    df['openinterest'] = 0 #设置Openinterest
    df = df[['open','high','low','close','vol','openinterest']]
    df.rename(columns={'vol': 'volume'}, inplace=True)
    return df

stock_df = get_data()
print(stock_df.head())
data = bt.feeds.PandasData(
    dataname=stock_df,
    fromdate=pd.to_datetime(start_date),
    todate=pd.to_datetime(end_date),
)

              open    high     low   close    volume  openinterest
trade_date                                                        
2017-01-03  334.28  337.00  332.81  334.56  20763.89             0
2017-01-04  334.62  352.17  334.60  351.91  65257.38             0
2017-01-05  350.00  351.45  345.44  346.74  41704.48             0
2017-01-06  346.64  359.78  346.10  350.76  68095.62             0
2017-01-09  347.80  352.88  346.54  348.51  35405.00             0


In [ ]:
class MovingAvg(bt.Strategy):
    # list of parameters which are configurable for the strategy
    params = dict(
        pfast=10,  # period for the fast moving average
        pslow=30   # period for the slow moving average
    )

    """
    params = (
        ('period_15',15),
        ('period_20',20),
    )
    """

    def __init__(self):
        self.order = None
        self.buyprice = None
        self.buycomm = None
        #self.sma = bt.indicators.SimpleMovingAverage(self.data0, period = self.params.pfast)
        self.sma = bt.indicators.SMA(self.datas[0], period = self.p.pfast)

    def next(self):
        if not self.position:  # not in the market
            if self.datas[0].close[0] > self.sma[0]:  # if fast crosses slow to the upside
                self.buy()  # enter long

        elif self.datas[0].close[0] < self.sma[0]:  # in the market & cross to the downside
            self.order = self.sell()  # close long position

In [6]:
# Create a subclass of Strategy to define the indicators and logic

class SmaCross(bt.Strategy):
    # list of parameters which are configurable for the strategy
    params = dict(
        pfast=10,  # period for the fast moving average
        pslow=30   # period for the slow moving average
    )

    def __init__(self):
        sma1 = bt.ind.SMA(period=self.p.pfast)  # fast moving average
        sma2 = bt.ind.SMA(period=self.p.pslow)  # slow moving average
        self.crossover = bt.ind.CrossOver(sma1, sma2)  # crossover signal

    def next(self):
        if not self.position:  # not in the market
            if self.crossover > 0:  # if fast crosses slow to the upside
                self.buy()  # enter long

        elif self.crossover < 0:  # in the market & cross to the downside
            self.close()  # close long position

In [22]:
# Create a subclass of Strategy to define the indicators and logic

class SmaCross(bt.Strategy):
    # list of parameters which are configurable for the strategy
    params = dict(
        pfast=10,  # period for the fast moving average
        pslow=30   # period for the slow moving average
    )

    def __init__(self):
        sma1 = bt.ind.SMA(period=self.p.pfast)  # fast moving average
        sma2 = bt.ind.SMA(period=self.p.pslow)  # slow moving average
        self.crossover = bt.ind.CrossOver(sma1, sma2)  # crossover signal

    def next(self):
        if not self.position:  # not in the market
            if self.crossover > 0:  # if fast crosses slow to the upside
                self.order_target_size(target=1)  # enter long

        elif self.crossover < 0:  # in the market & cross to the downside
            self.order_target_size(target=0)  # close long position

In [ ]:
# Create a subclass of SignaStrategy to define the indicators and signals

class SmaCross(bt.SignalStrategy):
    # list of parameters which are configurable for the strategy
    params = dict(
        pfast=10,  # period for the fast moving average
        pslow=30   # period for the slow moving average
    )

    def __init__(self):
        sma1 = bt.ind.SMA(period=self.p.pfast)  # fast moving average
        sma2 = bt.ind.SMA(period=self.p.pslow)  # slow moving average
        crossover = bt.ind.CrossOver(sma1, sma2)  # crossover signal
        self.signal_add(bt.SIGNAL_LONG, crossover)  # use it as LONG signal

In [14]:
cerebro = bt.Cerebro()  # create a "Cerebro" engine instance

cerebro.adddata(data)  # Add the data feed
cerebro.addstrategy(SmaCross)  # Add the trading strategy

cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='SharpeRatio')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='DrawDown')

cerebro.broker.setcash(1000000)
cerebro.broker.setcommission(commission=0.0005)

s = pd.to_datetime(start_date).strftime("%Y-%m-%d")
t = pd.to_datetime(end_date).strftime("%Y-%m-%d")

print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.addsizer(bt.sizers.PercentSizer, percents=90)
cerebro.run()  # run it all
print('Ending Portfolio Value: %.2f' % cerebro.broker.getvalue())

#cerebro.plot(iplot=False)[0][0]
#cerebro.plot(mpl={'style': 'mpl'})[0][0]
#%matplotlib inline

Starting Portfolio Value: 1000000.00
Ending Portfolio Value: 1686255.71
